<a href="https://colab.research.google.com/github/zoebatz/CAP6640_Project/blob/main/NLP_FLAN_T5_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece
!pip install sacrebleu
!pip install bert_score
!pip install evaluate
!pip install nltk
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
import os
import kagglehub
from bert_score import score
import sacrebleu
from nltk.translate.meteor_score import meteor_score
import nltk
import evaluate
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Download necessary resources for METEOR
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but recommended
nltk.download('punkt_tab')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# FLAN T5 for translation, summarization and QA

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
file_name = 'data_sum1_100'
path = "/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/"+file_name+".csv"
data = pd.read_csv(path)
data.head()

,text,summary,Dataset
0,Transportation officials in New York suspended...,"Avonte Oquendo, 14, has been missing since he ...",CNN/Daily Mail
1,By . Nathan Klein . Thousands of women migrant...,Women in Qatar are being exposed to 'utterly m...,CNN/Daily Mail
2,\n \n Paper Money Magazine's article about 19...,"– Sure, it says ""$10"" on the corner, but a Pen...",Multi-News
3,WASHINGTON (CNN) -- Look up at any presidentia...,Countersniper teams to be out in force across ...,CNN/Daily Mail
4,Bobby Brown Fighting Whitney's Family To See B...,– The investigation into Whitney Houston's dea...,Multi-News


In [17]:
len(data)

300

In [ ]:
'''# refresh drive
from google.colab import drive
drive.flush_and_unmount()

# mount drive
drive.mount('/content/drive')

SyntaxError: incomplete input (<ipython-input-6-e95900dfe7cc>, line 1)

In [20]:
# examples from full dataset not included in test data
prompt_examples = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_sum1_100_examples.csv')
#shuffle
prompt_examples = prompt_examples.sample(frac=1).reset_index(drop=True)
prompt_examples.head()

,text,summary,Dataset
0,"The feature is designed to be the ""modern day ...",You can't have a Facebook profile if you're un...,XSum
1,Story highlights Prison moves O.J. Simpson to ...,– Though his parole isn’t scheduled to start u...,Multi-News
2,Celtic striker John Guidetti has won the Scott...,John Guidetti is on loan from Manchester City ...,CNN/Daily Mail
3,Already garnering unwelcome attention for its ...,– The 18-year-old victim of a brutal beating b...,Multi-News
4,The session was due to take place at Huntingdo...,A health and safety training event has been ca...,XSum


In [19]:
len(prompt_examples)

300

In [21]:
import torch
# use gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [24]:
# small sample for testing
test_size = 10 #len(data)
data = data[:test_size]

predictions = [] # predicted translation

pred_zero = []
pred_one = []
pred_three = []

instruction_prompt = 'Summarize the following news articles to highlight the key facts. Provide answers to who, what, when, where, why and how in the summary in a natural way. The summary should be a few sentences long.'


In [25]:
batch_size = 1

function_name = 'summarize'
total = 0
j = 0


for i in range(0, test_size, batch_size):
    batch_inputs = data['text'].iloc[i:i+batch_size].tolist()


    # zero shot prompts
    zero_shot = [f"""{instruction_prompt}\n{text}""" for text in batch_inputs]

    # one shot prompt
    one_shot = [f"""{instruction_prompt} Follow the example below.

    Article: {prompt_examples['text'][j % len(prompt_examples)]}
    Summary: {prompt_examples['summary'][j % len(prompt_examples)]}

    Article: {text}
    Summary:""" for j, text in enumerate(batch_inputs)]
    j += 1

    # three shot prompt
    three_shot =  [f"""{instruction_prompt} Follow the examples below.

    Article: {prompt_examples['text'][j % len(prompt_examples)]}
    Summary: {prompt_examples['summary'][j % len(prompt_examples)]}

    Article: {prompt_examples['text'][j % len(prompt_examples) + 1]}
    Summary: {prompt_examples['summary'][j % len(prompt_examples)]}

    Article: {prompt_examples['text'][j % len(prompt_examples) + 2]}
    Summary: {prompt_examples['summary'][j % len(prompt_examples)]}

    Article: {text}
    Summary:""" for text in enumerate(batch_inputs)]
    j += 3

    # tokenize batch
    inputs_zero = tokenizer(zero_shot, return_tensors="pt", truncation=True, padding=True).to(device)
    inputs_one = tokenizer(one_shot, return_tensors="pt", truncation=True, padding=True).to(device)
    inputs_three = tokenizer(three_shot, return_tensors="pt", truncation=True, padding=True).to(device)

    # translate for each prompt
    outputs_zero = model.generate(
        input_ids=inputs_zero.input_ids,
        attention_mask=inputs_zero.attention_mask,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )

    outputs_one = model.generate(
        input_ids=inputs_one.input_ids,
        attention_mask=inputs_one.attention_mask,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )

    outputs_three = model.generate(
        input_ids=inputs_three.input_ids,
        attention_mask=inputs_three.attention_mask,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )

    total += batch_size
    print(total)

    # decode outputs
    decoded_zero = tokenizer.batch_decode(outputs_zero, skip_special_tokens=True)
    print(decoded_zero)
    decoded_one = tokenizer.batch_decode(outputs_one, skip_special_tokens=True)
    print(decoded_one)
    decoded_three = tokenizer.batch_decode(outputs_three, skip_special_tokens=True)
    print(decoded_three)

    pred_zero.extend(decoded_zero)
    pred_one.extend(decoded_one)
    pred_three.extend(decoded_three)





1
['Avonte Oquendo, a 14-year-old with autism, was last seen on surveillance video running out of Center Boulevard School in Long Island City on October 4.']
['Summary: Avonte Oquendo was last seen on surveillance video running out of Center Boulevard School in Long Island City on October 4.']
["– OJ Simpson has moved to a new cell and away from any inmates who might want to hurt or harass him in the months before he is set to be released. A Nevada parole board on Thursday decided Simpson should be freed after the former NFL star apologized for his role in a 2007 armed robbery, said he'd been a model prisoner and promised that he'd have no conflicts if released. Brooke K"]
2
["Amnesty International report titled 'My sleep is my break: Exploitation of domestic workers in Qatar' Has uncovered the appalling conditions women - primarily from the Philippines, Indonesia and Sri Lanka - face on a daily basis."]
["Thousands of women migrant domestic workers in Qatar are being severely exploite

In [10]:
print(f'{pred_zero[0]} \n{pred_one[0]} \n{pred_three[0]}')

Land Rover launched the Range Rover in 1970, describing it as the most versatile car in the world. It was designed to combine the comfort and on-road ability of a Rover saloon with the off-road skill of a Land Rover. Today, it remains the benchmark in luxury off-roaders. 
Sir Keir Starmer disagrees with those calling for time limits on how long a person can languish on bail without being charged. 
Val Thorens Sensations is a 384-room hotel in the French Alps. It has bars and beer taps on each floor and an outdoor Jacuzzi. The hotel is the first of its kind to be completely crowd-sourced.


In [11]:
data['text'][0]

"By . Paul Donnelley . The first Range Rover production car, which set the benchmark for luxury 4x4s, is expected to sell for more than £150,000 when it comes up at auction. Land Rover launched the Range Rover in 1970, describing it as the most versatile car in the world. It was designed to combine the comfort and on-road ability of a Rover saloon with the off-road skill of a Land Rover. Land Rover launched the Range Rover in 1970, describing it as the most versatile car in the world . Today, it remains the benchmark in luxury off-roaders and looks at home on a farm, outside the Ritz or in the car park of a Premier League football ground. And it was this two-door, green model with the registration YVB 151H, which started it all - and it is expected to prompt a fierce bidding war among enthusiasts and investors. The car was built between November 24, 1969 and December 17, 1969 before being registered on January 2, 1970. The car was designed to combine the comfort and on-road ability of 

In [12]:
data['text'][1]

'The Shakespeare Birthplace Trust said it had spent 18 months preparing the site at New Place, Stratford-upon-Avon, in Warwickshire, and had now commenced work.\nThe project will see the building of an exhibition centre, a new entrance and the restoration of the garden.\nThe work is planned to end by 2016, the anniversary of Shakespeare\'s death.\nThe trust has spent several months excavating Shakespeare\'s living quarters - an area that has never previously been examined.\nA contractor has now been appointed to carry out the rest of the work, including the restoration of the sunken Knot Garden.\nJulie Crawshaw, project manager, said: "After 18 months of meticulous planning and consulting, the hard work now begins.\n"With less than a year to go until opening, all hands are on deck to ensure the project runs to schedule, and work is carried out with minimum disruption."\nThe project is being supported by the Heritage Lottery Fund and Historic England.'

In [ ]:
data['summary']

,summary
0,On launch Land Rover described it as the most ...
1,A £5.25m project to transform the site of Shak...
2,A.P.C. founder Jean Touitou has apologized fo...
3,Warner Bros has announced a follow-up to upcom...
4,Tesco unveiled plans to turn a pub in Dorset a...
5,– Has the wave of Arab unrest finally reached ...
6,Dunstall Hall in Staffordshire was built with ...
7,Wayne Routledge scored twice as Swansea City s...
8,"He was seen at Porthleven, Cornwall on Monday ..."
9,A Brooklyn couple is welcoming the two kittens...


In [17]:
for i in prompt_examples['summary']:
  print(i)


Britain's former top prosecutor says time 'should not be a barrier' to justice .
Disagreed with those who call for time limits on how long a person can remain on bail without being charged .
Comes after a number of celebrities, including Rolf Harris and Stuart Hall, have been convicted of crimes committed decades ago .
Club Med: Val Thorens Sensations was designed via crowd-sourcing .
Facebook consultation decided from carpet colours to sports facilities .
Hotel includes a climbing wall and bars with beer taps on each floor .
Manchester City defender Gael Clichy is likely to be available after a back problem for Wednesday's FA Cup replay.
Hundreds of cartoonists from around the world have taken part in a competition in Iran attacking Donald Trump.
– Today in the ugly lawsuit department: Walmart and the Walton family are suing for the rights to photo negatives, some half-a-century old, in what has been classified as "a total David vs. Goliath situation." The story goes like this: Robert

In [ ]:
pred_zero

['Land Rover launched the Range Rover in 1970, describing it as the most versatile car in the world. It was designed to combine the comfort and on-road ability of a Rover saloon with the off-road skill of a Land Rover. Today, it remains the benchmark in luxury off-roaders.',
 'A £1m project to restore the birthplace of Shakespeare has begun.',
 "A.P.C. founder Jean Touitou has apologized for using 'ignorant and offensive' remarks to introduce the brand's fall 2015 menswear show in Paris this week. Mr Touitou held up a sign that read, 'Last Ni##@$ IN PARIS,' and explained that A.P.C's collaboration with Timberland is 'a very strong ghetto signifier'",
 'The second Fantastic Beasts film series has been confirmed by Warner Bros.',
 'Blue and Red logo is a great way to make a pub look like a shop.',
 'Saudi security forces fired on scores of protesters in the city of Qatif, witnesses and activists said.',
 'Dunstall Hall in Burton on Trent, Staffordshire, boasts 12 bedrooms, its own lake, 

In [27]:
# change model name \ to _
model_name = model_name.replace("/", "_")
model_name

'google_flan-t5-base'

In [30]:
references = data['summary'].tolist()

In [31]:
# view metrics and save
def view_and_save_metrics(prediction, reference, shot_name):
  # BLEU
  bleu = sacrebleu.corpus_bleu(prediction, [reference])
#  print(f"BLEU: {bleu.score}")

  # Tokenize textences for METEOR
  prediction_tokens = [nltk.word_tokenize(pred) for pred in prediction]
  reference_tokens = [[nltk.word_tokenize(ref)] for ref in reference]  # Note the double list for reference

  # METEOR (textence-level average)
  meteor_scores = [
      meteor_score(ref_tokens, pred_tokens)
      for ref_tokens, pred_tokens in zip(reference_tokens, prediction_tokens)
  ]
  avg_meteor = sum(meteor_scores) / len(meteor_scores)

  # ROUGE
  rouge = evaluate.load('rouge')
  rouge_scores = rouge.compute(predictions=prediction, references=reference)
#  print(f"ROUGE: {rouge_scores}")

  # BERT Score
  P_, R_, F1 = score(prediction, reference, lang="el")
  # print(f"Avg BERTScore F1: {F1.mean().item():.4f}")
  bert_score = F1.mean().item()

  metrics = {
      "bleu": bleu.score,
      "meteor": avg_meteor,
      "bert_score": bert_score,
      "model_name": model_name,
      "dataset": file_name,
      "shot": shot_name,
      "size": test_size
  }

  metrics.update(rouge_scores)
  '''
  # Print all metrics
  print("\n--- Evaluation Metrics ---")
  print(f"BLEU:   {bleu.score:.4f}")
  print(f"METEOR: {avg_meteor:.4f}")
  for k, v in rouge_scores.items():
      print(f"{k.upper()}: {v:.4f}")
  '''

  # save metrics to csv file
  metrics_df = pd.DataFrame([metrics])
  metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/metrics/summarization/metrics_'+model_name+'_'+file_name+'_'+shot_name+'.csv', index=False)
  return metrics_df

In [32]:
zero_shot_metrics = view_and_save_metrics(pred_zero, references, "zero_shot")
one_shot_metrics = view_and_save_metrics(pred_one, references, "one_shot")
three_shot_metrics = view_and_save_metrics(pred_three, references, "three_shot")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [33]:
zero_shot_metrics

,bleu,meteor,bert_score,model_name,dataset,shot,size,rouge1,rouge2,rougeL,rougeLsum
0,4.602462,0.205167,0.723645,google_flan-t5-base,data_sum1_100,zero_shot,10,0.326993,0.11829,0.209363,0.251573


In [34]:
one_shot_metrics

,bleu,meteor,bert_score,model_name,dataset,shot,size,rouge1,rouge2,rougeL,rougeLsum
0,0.122713,0.10469,0.669072,google_flan-t5-base,data_sum1_100,one_shot,10,0.166051,0.057707,0.139623,0.155438


In [35]:
three_shot_metrics

,bleu,meteor,bert_score,model_name,dataset,shot,size,rouge1,rouge2,rougeL,rougeLsum
0,0.136917,0.056737,0.632168,google_flan-t5-base,data_sum1_100,three_shot,10,0.097551,0.002247,0.056403,0.072725
